In [1]:
# Import libraries
from library.common import Database
import pathlib

import numpy as np
import pandas as pd
import os
#import matplotlib.pyplot as plt
# import seaborn as sns 
import altair as alt
import warnings
warnings.filterwarnings("ignore")
# from sklearn.linear_model import LinearRegression
# import statsmodels.api as sm

# Display all columns of pandas dataframes
pd.set_option('display.max_columns', None)

db = Database()
write_to_db = True

# Get the path to raw data files
# path = pathlib.Path.cwd().parent.joinpath(path_to_data)

Read the consolidated_data from Database

In [2]:
data2 = pd.read_sql('consolidated_data', db.config)

## Step 2: Exploratory Data Analysis: Carbon dioxide emission 1960-2020

### Observation 1: Total carbon dioxide emission is on increasing trend, but may decline when there were economic recessions

On global basis, carbon dioxide emission was about 10 billion ton in 1960.  It increased by over 2 times to 35 billion ton in 2020.  While carbon dioxide emission basically increased every year, we observed occasional year-on-year decrease when there were economic recessions.  For example, we observed the largest year-on-year decline in emission level over the past 60 years, at -5% in 2020 when many goverments imposed restrictions on domestic and international travels, cut operating hours of bars and restaurants, required office implement work-from-home arrangements, etc. to curb the spread of COVID-19 pandemic. As a result, many countries suffered the worst economic downturns since World War II.

In [3]:
data2_world=data2[data2['Country Code']=='WLD']
data2_CHN=data2[data2['Country Code']=='CHN']
data2_USA=data2[data2['Country Code']=='USA']
data2_world['CO2 Emission_lag1']=data2_world['CO2 Emission'].shift(1)
data2_world['CO2_Emission_YOY']=(data2_world['CO2 Emission']/data2_world['CO2 Emission_lag1']-1)*100

In [4]:
# Annual Global CO2 emission
chart1a=alt.Chart(data2_world).mark_line().encode(
    alt.X('Year:O', axis=alt.Axis(title='Year')),
    alt.Y('CO2 Emission:Q',
          axis=alt.Axis(title='CO2 Emission (thousand ton)')),
    color=alt.value('forestgreen')
).properties(width=400, height=300,
             title=alt.TitleParams(text='Annual Global CO2 Emission, 1960-2020',
                                   fontSize=15))

yoy=alt.Chart(data2_world).mark_bar().encode(
    alt.X('Year:O', axis=alt.Axis(title='Year')),
    alt.Y('CO2_Emission_YOY:Q',
          axis=alt.Axis(title='Year-on-year change in CO2 emission (%)')),
    color=alt.condition('datum.CO2_Emission_YOY>=0', alt.ColorValue('cornflowerblue'), alt.ColorValue('lightcoral'))
)
text1 = alt.Chart({'values':[{'y': -6.05}]}).mark_text(
    text='COVID-19 \npandemic',dx=180,lineBreak='\n',size=10
).encode(
    y='y:Q'
)
text2 = alt.Chart({'values':[{'y': -1.9}]}).mark_text(
    text='Global \nfinancial \ncrisis',dx=120,lineBreak='\n',size=10
).encode(
    y='y:Q'
)
text3 = alt.Chart({'values':[{'y': -3.3}]}).mark_text(
    text='Gulf war',dx=10,size=10
).encode(
    y='y:Q'
)
text4 = alt.Chart({'values':[{'y': -2.7}]}).mark_text(
    text='Energy crisis \nafter Iranian\nrevolution',dx=-65,lineBreak='\n',size=10
).encode(
    y='y:Q'
)
chart1b = alt.layer(yoy,text1,text2,text3,text4).resolve_scale(y='shared').properties(width=400, height=300,
             title=alt.TitleParams(text='Year-on-year % Change in Global CO2 Emission',
                                   fontSize=15))

chart1a|chart1b

alt.HConcatChart(...)

### Observation 2: Electricity and heat is the major source of carbon dioxide emission

Over the past three decades from 1990 to 2018, electricity and heat was the largest source of carbon dioxide emission and represented slightly over 40% of total carbon dioxide emitted.  Manufacturing and construction came next and represented about 15% of total emission.

In [5]:
chart_a=data2_world[(data2_world['Year']>=1990)&(data2_world['Year']<=2018)]
chart_a=chart_a[['Year','CO2 Emission','CO2 Emission_Electricity','CO2 Emission_Building','CO2 Emission_Manufacturing','CO2 Emission_Transport']]
chart_a['CO2 Emission_Other']=chart_a['CO2 Emission']-chart_a['CO2 Emission_Electricity']-chart_a['CO2 Emission_Building']-chart_a['CO2 Emission_Manufacturing']-chart_a['CO2 Emission_Transport']
chart_a.rename(columns={'CO2 Emission_Electricity':'Electricity & Heat','CO2 Emission_Building':'Building',
                        'CO2 Emission_Manufacturing':'Manufacturing & Construction','CO2 Emission_Transport':"Transport (excl. int'l bunkers)",
                        'CO2 Emission_Other':'Others'}, inplace=True)
chart_a.drop(['Year','CO2 Emission'],axis=1,inplace=True)
stacked_df=chart_a.stack().reset_index()
stacked_df.drop(['level_0'],axis=1,inplace=True)
stacked_df.rename(columns={'level_1':'Source',0:'Amount'}, inplace=True)

import itertools
lst = range(1990,2019)
year_lst=list(itertools.chain.from_iterable(itertools.repeat(x, 5) for x in lst))
stacked_df['Year']=year_lst

In [6]:
bars = alt.Chart(stacked_df).mark_bar().transform_joinaggregate(
    order='sum(Amount)',
    groupby=['Year']).encode(
    x=alt.X('sum(Amount):Q', stack='normalize', 
            axis=alt.Axis(title='Share of Emission Source (%)',format='%')),
    y=alt.Y('Year:O',
            axis=alt.Axis(title='Year')),
    color=alt.Color('Source:N',
                     scale=alt.Scale(domain=['Electricity & Heat', 'Manufacturing & Construction', 'Building', "Transport (excl. int'l bunkers)", 'Others'],
                     range=['orange','lightskyblue','lightyellow','mediumturquoise','thistle'])),
    order=alt.Order('order:Q',sort='descending')
                     
).properties(width=500, height=250,
             title=alt.TitleParams(text='Percentage Share of CO2 Emission by Source',dy=-5,fontSize=15)
            )

bars 

alt.Chart(...)

### Observation 3: Decreasing share of North America and increasing share of Asia Pacific in total carbon dioxide emission were observed

In 1960, emission from Europe and North America represented nearly 80% of total global emission.  Over the past 60 years, share of Europe and North America shrank gradually, while share of Asia picked up.  In 2020, emission from Europe and North America represented only slightly over 30% of total global emission.  while share of East Asia and South Asia rose to over 50%.

In [7]:
data2_60_20=data2[~(data2['Country Code']=='WLD')]
data2_60_20gp=data2_60_20.groupby(['Year','region']).agg({'CO2 Emission':'sum'}).reset_index()

In [8]:
# Annual CO2 emission by region (don't know how to swap order)
#site_order = ['Europe & Central Asia', 'North America', 'Sub-Saharan Africa', 'Latin America & Caribbean', 'Middle East & North Africa', 'South Asia', 'East Asia & Pacific']
chart2=alt.Chart(data2_60_20gp).transform_joinaggregate(
    order='min(CO2 Emission)',
    groupby=['region']).mark_area().encode(
           x="Year:O",
           y=alt.Y("CO2 Emission:Q",stack='normalize',
                   axis=alt.Axis(title='Share of CO2 Emission (%)',format='%')
                  ),
           color=alt.Color('region:N', 
                    scale = alt.Scale(domain=['Europe & Central Asia','North America','Sub-Saharan Africa','Latin America & Caribbean','Middle East & North Africa','South Asia','East Asia & Pacific'],
                                      range=['darkred','lightcoral','lightcyan','lightskyblue','cornflowerblue','royalblue','darkblue']),
                    legend = alt.Legend(title='Region')
                          ),
           order=alt.Order('order:Q', sort='ascending')
        ).properties(width=400, height=300,
                     title=alt.TitleParams(text='Percentage Share of CO2 Emission by Region',fontSize=15)
                    ) 
chart2

alt.Chart(...)

If we focus on the two largest emitting countries in the world, viz. the US and China, it will be easy to understand why share of North America decreased and share of Asia increased over the past six decades. In fact, the decreasing share of North America and increasing share of Asia basically reflected the decreasing share of the US and the increasing share of China over the same period.

During 1960 - 2005, the US was the largest emitting country in the world.  Emission from the US in 1960 was about 3 billion ton, represented 30% of global emission at that time.  Emission from the US in the next 45 years continued to climb up, but at a rate slower than the global average.  Consequently, share of US in the global emission gradually fell to about 20% in 2005, albeit the emission amount actually rose to over 6 billion ton.  Since then, emission amount from the US started to decrease. In 2020, US emitted slightly less than 5 billion ton or represented 14% of the global emission, and is the second largest carbon dioxide emitting country in the world.

Before 1980, emission amount from China was less than 1.5 billion ton each year. The Open Door Policy since 1980s attracted foreign investment and led to the first-stage expansion in China's economic ties with the West.  Carbon dioxide emission amount also started to pick up.  In 2000, emission amount from China already rose to about 3.5 billion ton, represented nearly 15% of global emission at that time.  The admission of China to World Trade Organization led to the second-stage expansion of China's economy. At the same time, carbon dioxide emission amount skyrocketed.  In only ten years, emission amount from China increased by 1.5 times and reached 9 billion ton in 2010, or over 25% of global emission at that time.  In fact, China has become the largest carbon dioxide emitting country in the world since 2006.  In 2020, emission amount from China was nearly 11 billion ton, represented over 30% of global emission.

In [9]:
# Annual CO2 emission - by two largest emitters (US & China)
data_chart3 = data2_USA.append(data2_CHN,ignore_index=True)
chart3a=alt.Chart(data_chart3).mark_line().encode(
    alt.X('Year:O', axis=alt.Axis(title='Year')),
    alt.Y('CO2 Emission:Q',
          axis=alt.Axis(title='CO2 Emission (thousand ton)')),
    color=alt.Color('Country Name:N',
                    scale = alt.Scale(domain=['China', 'United States'],
                                      range=['orange', 'cornflowerblue']),
                    legend = alt.Legend(title='Country')
                   )
).properties(width=380, height=300,
             title=alt.TitleParams(text='Annual CO2 Emission: US vs China',
                                   fontSize=15))

chart3b=alt.Chart(data_chart3).mark_line().encode(
    alt.X('Year:O', axis=alt.Axis(title='Year')),
    alt.Y('CO2 Country Share Percent:Q',
          axis=alt.Axis(title='Share of CO2 emission (%)')),
    color=alt.Color('Country Name:N',
                    scale = alt.Scale(domain=['China', 'United States'],
                                      range=['orange', 'cornflowerblue']),
                    legend = alt.Legend(title='Country')
                   )
).properties(width=380, height=300,
             title=alt.TitleParams(text='Percentage share of CO2 Emission: US vs China',
                                   fontSize=15))
chart3a|chart3b

alt.HConcatChart(...)

The sharp rise in total carbon dioxide emission of China since 2000 seems tally with an accelerated rise in various factors which drive carbon dioxide emission as mentioned in various literature, including a steeper growth in real-term economic size (as measured by GDP at constant dollars) and energy consumption (as measured by primary energy consumption, which includes consumption of the energy sector, losses during transformation and distribution of energy, and the final consumption by end users); the becoming of the 'world's factory' (as measured by an increasing share in the total value added generated from manufacturing activities in the world); and a slightly faster increase in share of urban population compared to the growths observed in 1990s.

On the contrary, we observed that the US has become a more energy-efficient economy in tandem with a similar level of carbon dioxide emission between 1990 and 2020.  While the whole US economy has broadly consumed similar amount of energy every year over the past 30 years, the real-term economic size has continued to grow steadily over the same period. In other words, the US is able to consume less amount of energy in order to produce the same dollar amount of GDP. 

In [10]:
data_chart4 = data_chart3[data_chart3['Year']>=1990]
data_chart4['Constant GDP']=data_chart4['Constant GDP']/1000000 #conver from $US to $US million
chart4a=alt.Chart(data_chart4).mark_line().encode(
    alt.X('Year:O', axis=alt.Axis(title='Year')),
    alt.Y('Constant GDP:Q',
          axis=alt.Axis(title='2015 $US million')),
    color=alt.Color('Country Name:N',
                    scale = alt.Scale(domain=['China', 'United States'],
                                      range=['orange', 'cornflowerblue']),
                    legend = alt.Legend(title='Country')
                   )
).properties(width=200, height=200,
             title=alt.TitleParams(text='Constant GDP',
                                   fontSize=15))

chart4b=alt.Chart(data_chart4).mark_line().encode(
    alt.X('Year:O', axis=alt.Axis(title='Year')),
    alt.Y('Manufacturing Country Share Percent:Q',
          axis=alt.Axis(title='Percent (%)')),
    color=alt.Color('Country Name:N',
                    scale = alt.Scale(domain=['China', 'United States'],
                                      range=['orange', 'cornflowerblue']),
                    legend = alt.Legend(title='Country')
                   )
).properties(width=200, height=200,
             title=alt.TitleParams(text='Share in Total Manufacturing',subtitle='Value Added of the World',
                                   fontSize=15,subtitleFontSize=15,subtitleFontWeight='bold'))

chart4c=alt.Chart(data_chart4).mark_line().encode(
    alt.X('Year:O', axis=alt.Axis(title='Year')),
    alt.Y('Primary Energy Consumption:Q',
          axis=alt.Axis(title='Petajoules')),
    color=alt.Color('Country Name:N',
                    scale = alt.Scale(domain=['China', 'United States'],
                                      range=['orange', 'cornflowerblue']),
                    legend = alt.Legend(title='Country')
                   )
).properties(width=200, height=200,
             title=alt.TitleParams(text='Primary Energy Consumption',
                                   fontSize=15))

chart4d=alt.Chart(data_chart4).mark_line().encode(
    alt.X('Year:O', axis=alt.Axis(title='Year')),
    alt.Y('Urban Population Percent:Q',
          axis=alt.Axis(title='Percent (%)')),
    color=alt.Color('Country Name:N',
                    scale = alt.Scale(domain=['China', 'United States'],
                                      range=['orange', 'cornflowerblue']),
                    legend = alt.Legend(title='Country')
                   )
).properties(width=200, height=200,
             title=alt.TitleParams(text='Share of Urban Population in',subtitle='Total Population of the Country',
                                   fontSize=15,subtitleFontSize=15,subtitleFontWeight='bold'))
chart4_1=chart4a|chart4b
chart4_2=chart4c|chart4d
chart4=alt.vconcat(chart4_1,chart4_2)
chart4

alt.VConcatChart(...)

Since many products manufactured in China are produced on behalf of Western brand names in order to take advantage of the more cost efficient supply chain in China, some people argued that some of the carbon dioxide emission recorded in developing countries (like China) should actually been counted as emission in developed countries (like the US) which finally consumed the products.  Consequently, some experts have computed the trade-adjusted 'consumption-based carbon dioxide emission'.

Although carbon dioxide emission from the US increased and emission from China decreased under the consumption-based emission definition, China was still the largest emitting country in the world and the US comes second.

In [11]:
path_to_data = r'data/raw/manifest'
path = str(pathlib.Path.cwd().parent.joinpath(path_to_data))+r"/"
table = 'owid_co2_greenhouse_gas_emissions'
co2_emission_owid = pd.read_sql(table, db.config)
co2_consumption_emission = co2_emission_owid[['iso_code','year','consumption_co2']]
co2_consumption_emission.rename(columns={'iso_code':'Country Code','year':'Year','consumption_co2':'Consumption CO2 Emission'}, inplace=True)
co2_consumption_emission['Consumption CO2 Emission']=co2_consumption_emission['Consumption CO2 Emission']*1000 # Convert emission to thousand ton
data_chart5 = pd.merge(data_chart4, co2_consumption_emission, how='left', on=['Country Code','Year']) 
data_chart5 = data_chart5[data_chart5['Year']<=2018]

In [12]:
chart5a=alt.Chart(data_chart5).mark_line().encode(
    alt.X('Year:O', axis=alt.Axis(title='Year')),
    alt.Y('Consumption CO2 Emission:Q',
          axis=alt.Axis(title='Thousand ton')),
    color=alt.Color('Country Name:N',
                    scale = alt.Scale(domain=['China', 'United States'],
                                      range=['orange', 'cornflowerblue']),
                    legend = alt.Legend(title='Country')
                   )
).properties(width=500, height=300,
             title=alt.TitleParams(text='Annual CO2 Emission: Consumption-based vs Production-based',
                                   subtitle='Consumption-based emission in solid line',
                                   fontSize=15,subtitleFontSize=12))

chart5b=alt.Chart(data_chart5).mark_line(strokeDash=[3,1]).encode(
    alt.X('Year:O', axis=alt.Axis(title='Year')),
    alt.Y('CO2 Emission:Q',
          axis=alt.Axis(title='CO2 emission (thousand ton)')),
    color=alt.Color('Country Name:N',
                    scale = alt.Scale(domain=['China', 'United States'],
                                      range=['orange', 'cornflowerblue']),
                    legend = alt.Legend(title='Country')
                   )
).properties(width=500, height=300,
             title=alt.TitleParams(text='Annual CO2 Emission: Consumption-based vs Production-based',
                                   subtitle='Consumption-based emission in solid line',
                                   fontSize=15,subtitleFontSize=14))

text1 = alt.Chart({'values':[{'y': 7300000}]}).mark_text(
    text='US emitted more \nunder consumption-based',lineBreak='\n',dx=-10,size=10
).encode(
    y='y:Q'
)

text2 = alt.Chart({'values':[{'y': 8200000}]}).mark_text(
    text='China emitted less \nunder consumption-based',lineBreak='\n',dx=200,size=10
).encode(
    y='y:Q'
)

alt.layer(chart5a, chart5b, text1, text2).resolve_scale(
    x = 'shared',
    y = 'shared'
)

alt.LayerChart(...)

### Observation 4: Different trends were observed between total carbon dioxide emission and carbon dioxide emission intensity

While total carbon dioxide emission increased by 2.5 times from 10 billion ton to 35 billion over 1960-2020, the trend looks slightly different if we study 'emission per capita' and 'emission per constant GDP dollar' (i.e., carbon dioxide emission intensity).  Specifically, when expressing emission on per capita basis, carbon dioxide emission increased only by 45% over the past 60 years, from about 3.0 ton per person to 4.5 ton per person in 2020.  If we express emission on intensity basis, carbon dioxide emission actually decreased by 50%, from about 0.9kg per constant GDP dollar to about 0.4kg per constant GDP dollar over the same period.  This means that we actually emit less carbon dioxide in order to generate one GDP dollar!

We have introduced above that China and the US are the largest and the second largest countries in terms of total emission.  In terms of emission per capita, we may guess that Western countries will generally have higher amount of emission per person. This is quite true, but when only considering countries with population greater than 2 million, the US actually ranked 8th in 2020.  Qatar was the country which had the highest amount of emission per person in 2020.  In fact, top countries which have the highest amount of emission per person are usually oil producing countries, including Qatar, Kuwait and Saudi Arabia.

Carbon dioxide emission intensity is a bit difficult to understand.  Generally speaking, developed countries have more advanced and efficient production technology as well as higher proportion of service sectors in GDP, so they will have lower emission intensities, meaning that they can generate less amount of carbon dioxide per unit of economic output.  On the contrary, developing countries have less efficient production technology and lower proportion of service sectors in GDP, so they will have higher emission intensities, meaning that they need to generate more carbon dioxide per unit of economic output.  In 2020, Mongolia was the country with the highest emission intensity, and Ukraine was the second largest country.

In [13]:
data2_2020=data2[data2['Year']==2020]
data2_2020=data2_2020[~(data2_2020['Country Code']=='WLD')]
data2_2020a=data2_2020.sort_values(by='CO2 Emission',ascending=False).head(20)
data2_2020p=data2_2020[data2_2020['Population']>=2000000]
data2_2020bp=data2_2020p.sort_values(by='CO2 Emission per capita',ascending=False).head(20)
data2_2020c=data2_2020.sort_values(by='CO2 Emission per constant GDP',ascending=False).head(20)

In [14]:
chart5a=alt.Chart(data2_world).mark_line().encode(
    alt.X('Year:O', axis=alt.Axis(title='Year')),
    alt.Y('CO2 Emission:Q',
          axis=alt.Axis(title='CO2 Emission (thousand ton)')),
    color=alt.value('forestgreen')
).properties(width=300, height=300,
             title=alt.TitleParams(text='Annual Global CO2 Emission, 1960-2020',
                                   fontSize=15))

chart5b=alt.Chart(data2_world).mark_line().encode(
    alt.X('Year:O', axis=alt.Axis(title='Year')),
    alt.Y('CO2 Emission per capita:Q',
          axis=alt.Axis(title='CO2 Emission (ton per person)')),
    color=alt.value('forestgreen')
).properties(width=300, height=300,
             title=alt.TitleParams(text='Global CO2 Emission per capita, 1960-2020',
                                   fontSize=15))

chart5c=alt.Chart(data2_world).mark_line().encode(
    alt.X('Year:O', axis=alt.Axis(title='Year')),
    alt.Y('CO2 Emission per constant GDP:Q',
          axis=alt.Axis(title='CO2 Emission (kg per $GDP (2015 $US))')),
    color=alt.value('forestgreen')
).properties(width=300, height=300,
             title=alt.TitleParams(text='Global CO2 Emission Intensity, 1960-2020',
                                   fontSize=15))

In [15]:
# Annual Global CO2 emission and CO2 emission per capita - Top 10 countries in 2019
chart5a2 = alt.Chart(data2_2020a).mark_bar().encode(
    x=alt.X('CO2 Emission:Q',
            title='CO2 Emission (thousand ton)',
            axis=alt.Axis(titleFontSize=12,labelFontSize=10)),
    y=alt.Y('Country Name:N',
            sort='-x',
            title='',
            axis=alt.Axis(titleFontSize=12,labelFontSize=12)),
    color=alt.Color('income_group:N', 
                    scale = alt.Scale(domain=['High income', 'Upper middle income', 'Lower middle income', 'Low income'],
                                      range=['navy', 'cornflowerblue', 'lightcoral', 'darkred']),
                    legend = alt.Legend(title='Income Group'))
).properties(width=250, height=300,
            title=alt.TitleParams(text='Annual CO2 Emission 2020',
                                   subtitle='The 20 countries with the highest amount of emission', subtitleFontSize=13,
                                   fontSize=15,dy=-5))

chart5b2 = alt.Chart(data2_2020bp).mark_bar().encode(
    x=alt.X('CO2 Emission per capita:Q',
            title='CO2 emission per capita (ton per person)',
            axis=alt.Axis(titleFontSize=12,labelFontSize=12)),
    y=alt.Y('Country Name:N',
            sort='-x',
            title='',
            axis=alt.Axis(titleFontSize=12,labelFontSize=12)),
    color=alt.Color('income_group:N', 
                    scale = alt.Scale(domain=['High income', 'Upper middle income', 'Lower middle income', 'Low income'],
                                      range=['navy', 'cornflowerblue', 'lightcoral', 'darkred']),
                    legend = alt.Legend(title='Income Group'))
).properties(width=250, height=300,
            title=alt.TitleParams(text='CO2 Emission per capita 2020',
                                   subtitle='The 20 countries with the highest amount of emission', subtitleFontSize=13,
                                   fontSize=15,dy=-5))


chart5c2 = alt.Chart(data2_2020c).mark_bar().encode(
    x=alt.X('CO2 Emission per constant GDP:Q',
            title='CO2 emission intensity (kg per $GDP (2015 $US))',
            axis=alt.Axis(titleFontSize=12,labelFontSize=12)),
    y=alt.Y('Country Name:N',
            sort='-x',
            title='',
            axis=alt.Axis(titleFontSize=12,labelFontSize=12)),
    color=alt.Color('income_group:N', 
                    scale = alt.Scale(domain=['High income', 'Upper middle income', 'Lower middle income', 'Low income'],
                                      range=['navy', 'cornflowerblue', 'lightcoral', 'darkred']),
                    legend = alt.Legend(title='Income Group'))
).properties(width=250, height=300,
            title=alt.TitleParams(text='CO2 Emission Intensity 2020',
                                   subtitle='The 20 countries with the highest intensity', subtitleFontSize=13,
                                   fontSize=15,dy=-5))
chart5_1=chart5a|chart5a2
chart5_2=chart5b|chart5b2
chart5_3=chart5c|chart5c2
chart5=alt.vconcat(chart5_1,chart5_2,chart5_3)
chart5

alt.VConcatChart(...)

A SPLOM plot is prepared for each of the three measures of carbon dioxide emission (viz. total, per capita and intensity) with the features which should have impact on carbon dioxide emission (viz. GDP per capita (as proxy of income), energy intensity (i.e., primary energy consumption per constant GDP dollar), percentage of urban population, percentage of the country's manufacturing value added in the world's total, trade openness (viz. share of import and export of goods and services in the country's GDP), percentage share of renewable energy consumption over the country's primary energy consumption, and percentage share of environmental-related patents over total number of patents issued in the country.

### Observation 5a: Correlation between carbon dioxide emission per capita with various input features

We observed a moderate correlation between carbon dioxide emission per capita and:
* Constant GDP per capita (as a proxy of income): +0.62
* Urban population percentage: +0.51
* Share of renewable energy consumption in total energy consumption: -0.47

In [16]:
# Correlation plot between CO2 per constant GDP vs Energy per capita
data2_corr = data2[~(data2['Country Code']=='WLD')]
data2_corr = data2[['Year','CO2 Emission','CO2 Emission per capita','CO2 Emission per constant GDP','Constant GDP per capita','Energy Intensity','Urban Population Percent','Trade Openness','Renewable Energy Consumption Share','Percent of Environment Patent','income_group']]
data2_corr.dropna(inplace=True)
data2_corr_pc = data2_corr[['CO2 Emission per capita','Constant GDP per capita','Energy Intensity','Urban Population Percent','Trade Openness','Renewable Energy Consumption Share','Percent of Environment Patent']]
data2_corr_in = data2_corr[['CO2 Emission per constant GDP','Constant GDP per capita','Energy Intensity','Urban Population Percent','Trade Openness','Renewable Energy Consumption Share','Percent of Environment Patent']]
data2_corr_tot = data2_corr[['CO2 Emission','Constant GDP per capita','Energy Intensity','Urban Population Percent','Trade Openness','Renewable Energy Consumption Share','Percent of Environment Patent']]
#data2_corr1=data2_corr[(data2_corr['Year']>=1965)&(data2_corr['Year']<=1989)]
#data2_corr2=data2_corr[(data2_corr['Year']>=1990)&(data2_corr['Year']<=2020)]

In [17]:
data2_corr_pc.corr()

,CO2 Emission per capita,Constant GDP per capita,Energy Intensity,Urban Population Percent,Trade Openness,Renewable Energy Consumption Share,Percent of Environment Patent
CO2 Emission per capita,1.000000,0.616076,-0.098504,0.513791,0.199926,-0.470460,-0.133660
Constant GDP per capita,0.616076,1.000000,-0.444606,0.556912,0.298702,-0.200559,-0.222129
Energy Intensity,-0.098504,-0.444606,1.000000,-0.362287,-0.073953,0.069665,0.208996
Urban Population Percent,0.513791,0.556912,-0.362287,1.000000,0.236823,-0.472228,-0.215739
Trade Openness,0.199926,0.298702,-0.073953,0.236823,1.000000,-0.222586,-0.019618
Renewable Energy Consumption Share,-0.470460,-0.200559,0.069665,-0.472228,-0.222586,1.000000,0.236962
Percent of Environment Patent,-0.133660,-0.222129,0.208996,-0.215739,-0.019618,0.236962,1.000000


In [18]:
base = alt.Chart(data2_corr).transform_fold(
    ['CO2 Emission per capita', 'Constant GDP per capita', 'Energy Intensity','Urban Population Percent','Trade Openness','Renewable Energy Consumption Share','Percent of Environment Patent'],
    as_=['key_x', 'value_x']
).transform_fold(
    ['CO2 Emission per capita', 'Constant GDP per capita', 'Energy Intensity','Urban Population Percent','Trade Openness','Renewable Energy Consumption Share','Percent of Environment Patent'],
    as_=['key_y', 'value_y']
).encode(
    x=alt.X('value_x:Q', title=None),
    y=alt.Y('value_y:Q', title=None),
).properties(
    width=150,
    height=150
)

alt.layer(
    base.mark_circle().encode(color = alt.Color('income_group:N',
                              scale = alt.Scale(domain=['High income', 'Upper middle income', 'Lower middle income', 'Low income'],
                                      range=['navy', 'cornflowerblue', 'lightcoral', 'darkred']),
                              legend = alt.Legend(title='Income Group')
                              )),
    base.transform_regression(
        'value_x', 'value_y',
        groupby=['key_x', 'key_y', 'income_group']
    ).mark_line(
        color='orange'
    ).encode(
        detail='income_group:N'
    )
).facet(
    column=alt.Column('key_x:N', title=None),
    row=alt.Row('key_y:N', sort='descending')
).resolve_scale(
    x='independent',
    y='independent'
).properties(title=alt.TitleParams(text=['Correlation Plot among CO2 Emission per capita, Constant GDP per capita, Energy Intensity, Percentage of Urban Population','Trade Openness, Share of Renewable Energy in Total Energy Consumption and Share of Environment Patents in Total Patents Issued'], 
                                         anchor='middle', fontSize=15, dy=-5))

AttributeError: 'list' object has no attribute 'get'

### Observation 5b: Correlation between carbon dioxide emission intensity with various input features¶

We observed a weak to moderate correlation between carbon dioxide emission intensity and share of renewable energy consumption / constant GDP per capita, and a strong correlation between emission intensity and energy intensity:

* Energy intensity: +0.88
* Share of renewable energy consumption in total energy consumption: -0.26
* Constant GDP per capita: -0.40

In [ ]:
data2_corr_in.corr()

In [ ]:
base = alt.Chart(data2_corr).transform_fold(
    ['CO2 Emission per constant GDP', 'Constant GDP per capita', 'Energy Intensity','Urban Population Percent','Trade Openness','Renewable Energy Consumption Share', 'Percent of Environment Patent'],
    as_=['key_x', 'value_x']
).transform_fold(
    ['CO2 Emission per constant GDP', 'Constant GDP per capita', 'Energy Intensity','Urban Population Percent','Trade Openness','Renewable Energy Consumption Share', 'Percent of Environment Patent'],
    as_=['key_y', 'value_y']
).encode(
    x=alt.X('value_x:Q', title=None),
    y=alt.Y('value_y:Q', title=None),
).properties(
    width=150,
    height=150
)

alt.layer(
    base.mark_circle().encode(color = alt.Color('income_group:N',
                              scale = alt.Scale(domain=['High income', 'Upper middle income', 'Lower middle income', 'Low income'],
                                      range=['navy', 'cornflowerblue', 'lightcoral', 'darkred']),
                              legend = alt.Legend(title='Income Group')
                              )),
    base.transform_regression(
        'value_x', 'value_y',
        groupby=['key_x', 'key_y', 'income_group']
    ).mark_line(
        color='orange'
    ).encode(
        detail='income_group:N'
    )
).facet(
    column=alt.Column('key_x:N', title=None),
    row=alt.Row('key_y:N', sort='descending')
).resolve_scale(
    x='independent',
    y='independent'
).properties(title=alt.TitleParams(text=['Correlation Plot among CO2 Emission Intensity, Constant GDP per capita, Energy Intensity, Percentage of Urban Population','Trade Openness, Share of Renewable Energy in Total Energy Consumption and Share of Environment Patents in Total Patents Issued'], 
                                         anchor='middle', fontSize=15, dy=-5))

### Observation 5c: Correlation between total carbon dioxide emission with various input features
Correlation between total carbon dioxide emission and the features identified is not strong.  It seems that the correlation is masked by vast differences in population sizes among different countries.

In [ ]:
data2_corr_tot.corr()

In [ ]:
base = alt.Chart(data2_corr).transform_fold(
    ['CO2 Emission', 'Constant GDP per capita', 'Energy Intensity','Urban Population Percent','Trade Openness','Renewable Energy Consumption Share','Percent of Environment Patent'],
    as_=['key_x', 'value_x']
).transform_fold(
    ['CO2 Emission', 'Constant GDP per capita', 'Energy Intensity','Urban Population Percent','Trade Openness','Renewable Energy Consumption Share','Percent of Environment Patent'],
    as_=['key_y', 'value_y']
).encode(
    x=alt.X('value_x:Q', title=None),
    y=alt.Y('value_y:Q', title=None),
).properties(
    width=150,
    height=150
)

alt.layer(
    base.mark_circle().encode(color = alt.Color('income_group:N',
                              scale = alt.Scale(domain=['High income', 'Upper middle income', 'Lower middle income', 'Low income'],
                                      range=['navy', 'cornflowerblue', 'lightcoral', 'darkred']),
                              legend = alt.Legend(title='Income Group')
                              )),
    base.transform_regression(
        'value_x', 'value_y',
        groupby=['key_x', 'key_y', 'income_group']
    ).mark_line(
        color='orange'
    ).encode(
        detail='income_group:N'
    )
).facet(
    column=alt.Column('key_x:N', title=None),
    row=alt.Row('key_y:N', sort='descending')
).resolve_scale(
    x='independent',
    y='independent'
).properties(title=alt.TitleParams(text=['Correlation Plot among Annual CO2 Emission, Constant GDP per capita, Energy Intensity, Percentage of Urban Population','Trade Openness, Share of Renewable Energy in Total Energy Consumption and Share of Environment Patents in Total Patents Issued'], 
                                         anchor='middle', fontSize=15, dy=-5))

In [ ]:
#Extract data for PVAR
#some_value_1=['AUS', 'AUT', 'BEL', 'CHL', 'COL', 'DNK', 'ESP', 'FIN', 'FRA', 'GBR', 'GRC', 'IRL', 'ITA', 'JPN', 'KOR', 'LUX', 'MEX', 'NLD', 'NOR', 'PRT', 'SWE', 'TUR', 'USA']
#some_value_2=['AUS', 'AUT', 'BEL', 'CHL', 'COL', 'DNK', 'ESP', 'FIN', 'FRA', 'GBR', 'GRC', 'IRL', 'ITA', 'JPN', 'KOR', 'LUX', 'MEX', 'NLD', 'NOR', 'PRT', 'SWE', 'TUR', 'USA','ZAF', 'THA', 'SGP', 'NZL', 'MAR', 'IDN', 'IND', 'DEU', 'BRA', 'CHN', 'CHE']
#df_pvar=data2[['Country Name','Country Code','Year','CO2 Emission per constant GDP','Real GDP Growth %','Final Energy Consumption per capita','Trade Openness','Energy Use per capita']]
df_pvar_test = data2[['Country Name','Country Code','Year','CO2 Emission per constant GDP','Constant GDP per capita','Trade Openness','Primary Energy Consumption per capita','region']]
#df_pvar_test = df_pvar_test[~(df_pvar_test['Year']==2019)&~(df_pvar_test['Year']==2020)]
df_pvar_test = df_pvar_test[(df_pvar_test['Year']>=1990)&(df_pvar_test['Year']<=2014)]
df_pvar_test = df_pvar_test[~(df_pvar_test['region']=='North America')&~(df_pvar_test['region']=='Europe & Central Asia')]
df_pvar_test = df_pvar_test[~(df_pvar_test['Country Code']=='WLD')]
df_pvar_test = df_pvar_test.dropna(subset=['CO2 Emission per constant GDP','Constant GDP per capita','Trade Openness','Primary Energy Consumption per capita'], how='any')
df_pvar_test.to_csv('pvar_test.csv')
#df_pvar = df_pvar[~(df_pvar['Year']==2019)&~(df_pvar['Year']==2020)]
#df_pvar = df_pvar[~(df_pvar['Country Name']=='World')]
#df_pvar = df_pvar[(df_pvar['Year']>=1971)]
#df_pvar_test['CO2 Emission as Percent of GDP']=(df_pvar_test['CO2 Emission per constant GDP']/df_pvar_test['Constant GDP'])*100
#df1_pvar = df_pvar[(df_pvar['Country Code'].isin(some_value_1))]
#df2_pvar = df_pvar[(df_pvar['Year']>=1990)&(df_pvar['Country Code'].isin(some_value_2))]
#df_pvar.to_csv('pvar.csv')
#df_pvar_test = df_pvar_test[['Country Name','Country Code','Year','CO2 Emission as Percent of GDP','Real GDP Growth %','Trade Openness','Energy Use per capita']]
df_pvar_test2 = data2[['Country Name','Country Code','Year','CO2 Emission per constant GDP','Constant GDP per capita','Trade Openness','Primary Energy Consumption per capita','region']]
df_pvar_test2 = df_pvar_test2[(df_pvar_test2['region']=='North America')|(df_pvar_test2['region']=='Europe & Central Asia')]
df_pvar_test2 = df_pvar_test2[(df_pvar_test2['Year']>=1990)&(df_pvar_test2['Year']<=2014)]
df_pvar_test2 = df_pvar_test2.dropna(subset=['CO2 Emission per constant GDP','Constant GDP per capita','Trade Openness','Primary Energy Consumption per capita'], how='any')
df_pvar_test2.to_csv('pvar_test2.csv')
#df1_pvar.to_csv('pvar2a.csv')
#df2_pvar.to_csv('pvar2b.csv')
df_pvar_test3 = data2[['Country Name','Country Code','Year','CO2 Emission per constant GDP','Constant GDP per capita','Trade Openness','Primary Energy Consumption per capita','region']]
df_pvar_test3 = df_pvar_test3[(df_pvar_test3['Year']>=1990)&(df_pvar_test3['Year']<=2014)]
df_pvar_test3 = df_pvar_test3[~(df_pvar_test3['Country Code']=='WLD')]
df_pvar_test3 = df_pvar_test3.dropna(subset=['CO2 Emission per constant GDP','Constant GDP per capita','Trade Openness','Primary Energy Consumption per capita'], how='any')
df_pvar_test3.to_csv('pvar_test3.csv')

df_pvar_test4 = data2[['Country Name','Country Code','Year','CO2 Emission per constant GDP','Constant GDP per capita','Trade Openness','Primary Energy Consumption per capita','region']]
df_pvar_test4 = df_pvar_test4[~(df_pvar_test4['Country Code']=='WLD')]
df_pvar_test4 = df_pvar_test4.dropna(subset=['CO2 Emission per constant GDP','Constant GDP per capita','Trade Openness','Primary Energy Consumption per capita'], how='any')
df_pvar_test4.to_csv('pvar_test4.csv')

df_pvar_test5 = data2[['Country Name','Country Code','Year','CO2 Emission per constant GDP','Constant GDP per capita','Trade Openness','Primary Energy Consumption per capita','region']]
df_pvar_test5 = df_pvar_test5[~(df_pvar_test5['region']=='North America')&~(df_pvar_test5['region']=='Europe & Central Asia')]
df_pvar_test5 = df_pvar_test5[~(df_pvar_test5['Country Code']=='WLD')]
df_pvar_test5 = df_pvar_test5.dropna(subset=['CO2 Emission per constant GDP','Constant GDP per capita','Trade Openness','Primary Energy Consumption per capita'], how='any')
df_pvar_test5.to_csv('pvar_test5.csv')

df_pvar_test6 = data2[['Country Name','Country Code','Year','CO2 Emission per constant GDP','Constant GDP per capita','Trade Openness','Primary Energy Consumption per capita','region']]
df_pvar_test6 = df_pvar_test6[(df_pvar_test6['region']=='North America')|(df_pvar_test6['region']=='Europe & Central Asia')]
df_pvar_test6 = df_pvar_test6.dropna(subset=['CO2 Emission per constant GDP','Constant GDP per capita','Trade Openness','Primary Energy Consumption per capita'], how='any')
df_pvar_test6.to_csv('pvar_test6.csv')
#df_pvar_test

### Supplementary analysis #1: Index decomposition of China and US annual CO2 emission into four factors: 
* Population
* Income (GDP per capita)
* Energy intensity (primary energy consumed per constant GDP dollar); and 
* Fuel/technology mix (carbon dioxide emission per unit of primary energy consumed)

In [ ]:
# Index decomposition-China
data2_CHN_IDX = data2_CHN[['Year','CO2 Emission','Population','Constant GDP','Constant GDP per capita','Primary Energy Consumption']]
data2_CHN_IDX['CO2 Emission per unit Energy']=data2_CHN_IDX['CO2 Emission']/data2_CHN_IDX['Primary Energy Consumption'] # thousand tonne per PJ
data2_CHN_IDX['Energy Intensity']=(data2_CHN_IDX['Primary Energy Consumption'])/data2_CHN_IDX['Constant GDP'] # PJ per constant $GDP (2010 US dollar)
data2_CHN_IDX['Population_lag1']=data2_CHN_IDX['Population'].shift(1)
data2_CHN_IDX['Constant GDP per capita_lag1']=data2_CHN_IDX['Constant GDP per capita'].shift(1)
data2_CHN_IDX['Energy Intensity_lag1']=data2_CHN_IDX['Energy Intensity'].shift(1)
data2_CHN_IDX['CO2 Emission per unit Energy_lag1']=data2_CHN_IDX['CO2 Emission per unit Energy'].shift(1)
data2_CHN_IDX['CO2 Emission_lag1']=data2_CHN_IDX['CO2 Emission'].shift(1)
data2_CHN_IDX['CO2 Change'] = data2_CHN_IDX['CO2 Emission'] - data2_CHN_IDX['CO2 Emission_lag1']
data2_CHN_IDX['CO2 Emission per unit Energy Change'] = data2_CHN_IDX['CO2 Emission per unit Energy'] - data2_CHN_IDX['CO2 Emission per unit Energy_lag1']
data2_CHN_IDX['Energy Intensity Change'] = data2_CHN_IDX['Energy Intensity'] - data2_CHN_IDX['Energy Intensity_lag1']
data2_CHN_IDX['Constant GDP per capita Change'] = data2_CHN_IDX['Constant GDP per capita'] - data2_CHN_IDX['Constant GDP per capita_lag1']
data2_CHN_IDX['Population Change'] = data2_CHN_IDX['Population'] - data2_CHN_IDX['Population_lag1']
data2_CHN_IDX = data2_CHN_IDX[(data2_CHN_IDX['Year']>=1972)]
data2_CHN_IDX['Fuel Mix Effect']=(1/2)*(data2_CHN_IDX['CO2 Emission per unit Energy Change']*data2_CHN_IDX['Energy Intensity']*data2_CHN_IDX['Constant GDP per capita']*data2_CHN_IDX['Population']+data2_CHN_IDX['CO2 Emission per unit Energy Change']*data2_CHN_IDX['Energy Intensity_lag1']*data2_CHN_IDX['Constant GDP per capita_lag1']*data2_CHN_IDX['Population_lag1'])
data2_CHN_IDX['Energy Intensity Effect']=(1/2)*(data2_CHN_IDX['CO2 Emission per unit Energy_lag1']*data2_CHN_IDX['Energy Intensity Change']*data2_CHN_IDX['Constant GDP per capita']*data2_CHN_IDX['Population']+data2_CHN_IDX['CO2 Emission per unit Energy']*data2_CHN_IDX['Energy Intensity Change']*data2_CHN_IDX['Constant GDP per capita_lag1']*data2_CHN_IDX['Population_lag1'])
data2_CHN_IDX['Income Effect']=(1/2)*(data2_CHN_IDX['CO2 Emission per unit Energy_lag1']*data2_CHN_IDX['Energy Intensity_lag1']*data2_CHN_IDX['Constant GDP per capita Change']*data2_CHN_IDX['Population']+data2_CHN_IDX['CO2 Emission per unit Energy']*data2_CHN_IDX['Energy Intensity']*data2_CHN_IDX['Constant GDP per capita Change']*data2_CHN_IDX['Population_lag1'])
data2_CHN_IDX['Population Effect']=(1/2)*(data2_CHN_IDX['CO2 Emission per unit Energy_lag1']*data2_CHN_IDX['Energy Intensity_lag1']*data2_CHN_IDX['Constant GDP per capita_lag1']*data2_CHN_IDX['Population Change']+data2_CHN_IDX['CO2 Emission per unit Energy']*data2_CHN_IDX['Energy Intensity']*data2_CHN_IDX['Constant GDP per capita']*data2_CHN_IDX['Population Change'])
data2_CHN_IDXb=data2_CHN_IDX[['Fuel Mix Effect','Energy Intensity Effect','Income Effect','Population Effect']]
data2_CHN_IDX['Test']=data2_CHN_IDX['CO2 Change']-(data2_CHN_IDX['Fuel Mix Effect']+data2_CHN_IDX['Energy Intensity Effect']+data2_CHN_IDX['Income Effect']+data2_CHN_IDX['Population Effect'])
data2_CHN_IDXb = data2_CHN_IDXb.stack().reset_index()
data2_CHN_IDXb.drop(['level_0'],axis=1,inplace=True)
data2_CHN_IDXb.columns = ['Effect', 'CO2 Emission']
data2_CHN_IDXb['Year']=[
                       1972,1972,1972,1972,
                       1973,1973,1973,1973,
                       1974,1974,1974,1974,
                       1975,1975,1975,1975,
                       1976,1976,1976,1976,
                       1977,1977,1977,1977,
                       1978,1978,1978,1978,
                       1979,1979,1979,1979,
                       1980,1980,1980,1980,
                       1981,1981,1981,1981,
                       1982,1982,1982,1982,
                       1983,1983,1983,1983,
                       1984,1984,1984,1984,
                       1985,1985,1985,1985,
                       1986,1986,1986,1986,
                       1987,1987,1987,1987,
                       1988,1988,1988,1988,
                       1989,1989,1989,1989,
                       1990,1990,1990,1990,
                       1991,1991,1991,1991,
                       1992,1992,1992,1992,
                       1993,1993,1993,1993,
                       1994,1994,1994,1994,
                       1995,1995,1995,1995,
                       1996,1996,1996,1996,
                       1997,1997,1997,1997,
                       1998,1998,1998,1998,
                       1999,1999,1999,1999,
                       2000,2000,2000,2000,
                       2001,2001,2001,2001,
                       2002,2002,2002,2002,
                       2003,2003,2003,2003,
                       2004,2004,2004,2004,
                       2005,2005,2005,2005,
                       2006,2006,2006,2006,
                       2007,2007,2007,2007,
                       2008,2008,2008,2008,
                       2009,2009,2009,2009,
                       2010,2010,2010,2010,
                       2011,2011,2011,2011,
                       2012,2012,2012,2012,
                       2013,2013,2013,2013,
                       2014,2014,2014,2014,
                       2015,2015,2015,2015,
                       2016,2016,2016,2016,
                       2017,2017,2017,2017,
                       2018,2018,2018,2018,
                       2019,2019,2019,2019,
                       2020,2020,2020,2020
                      ]

In [ ]:
# Index decomposition-US
data2_USA_IDX = data2_USA[['Year','CO2 Emission','Population','Constant GDP','Constant GDP per capita','Primary Energy Consumption']]
data2_USA_IDX['CO2 Emission per unit Energy']=data2_USA_IDX['CO2 Emission']/data2_USA_IDX['Primary Energy Consumption'] # thousand tonne per PJ
data2_USA_IDX['Energy Intensity']=(data2_USA_IDX['Primary Energy Consumption'])/data2_USA_IDX['Constant GDP'] # PJ per constant $GDP (2010 US dollar)
data2_USA_IDX['Population_lag1']=data2_USA_IDX['Population'].shift(1)
data2_USA_IDX['Constant GDP per capita_lag1']=data2_USA_IDX['Constant GDP per capita'].shift(1)
data2_USA_IDX['Energy Intensity_lag1']=data2_USA_IDX['Energy Intensity'].shift(1)
data2_USA_IDX['CO2 Emission per unit Energy_lag1']=data2_USA_IDX['CO2 Emission per unit Energy'].shift(1)
data2_USA_IDX['CO2 Emission_lag1']=data2_USA_IDX['CO2 Emission'].shift(1)
data2_USA_IDX['CO2 Change'] = data2_USA_IDX['CO2 Emission'] - data2_USA_IDX['CO2 Emission_lag1']
data2_USA_IDX['CO2 Emission per unit Energy Change'] = data2_USA_IDX['CO2 Emission per unit Energy'] - data2_USA_IDX['CO2 Emission per unit Energy_lag1']
data2_USA_IDX['Energy Intensity Change'] = data2_USA_IDX['Energy Intensity'] - data2_USA_IDX['Energy Intensity_lag1']
data2_USA_IDX['Constant GDP per capita Change'] = data2_USA_IDX['Constant GDP per capita'] - data2_USA_IDX['Constant GDP per capita_lag1']
data2_USA_IDX['Population Change'] = data2_USA_IDX['Population'] - data2_USA_IDX['Population_lag1']
data2_USA_IDX = data2_USA_IDX[(data2_USA_IDX['Year']>=1961)]
data2_USA_IDX['Fuel Mix Effect']=(1/2)*(data2_USA_IDX['CO2 Emission per unit Energy Change']*data2_USA_IDX['Energy Intensity']*data2_USA_IDX['Constant GDP per capita']*data2_USA_IDX['Population']+data2_USA_IDX['CO2 Emission per unit Energy Change']*data2_USA_IDX['Energy Intensity_lag1']*data2_USA_IDX['Constant GDP per capita_lag1']*data2_USA_IDX['Population_lag1'])
data2_USA_IDX['Energy Intensity Effect']=(1/2)*(data2_USA_IDX['CO2 Emission per unit Energy_lag1']*data2_USA_IDX['Energy Intensity Change']*data2_USA_IDX['Constant GDP per capita']*data2_USA_IDX['Population']+data2_USA_IDX['CO2 Emission per unit Energy']*data2_USA_IDX['Energy Intensity Change']*data2_USA_IDX['Constant GDP per capita_lag1']*data2_USA_IDX['Population_lag1'])
data2_USA_IDX['Income Effect']=(1/2)*(data2_USA_IDX['CO2 Emission per unit Energy_lag1']*data2_USA_IDX['Energy Intensity_lag1']*data2_USA_IDX['Constant GDP per capita Change']*data2_USA_IDX['Population']+data2_USA_IDX['CO2 Emission per unit Energy']*data2_USA_IDX['Energy Intensity']*data2_USA_IDX['Constant GDP per capita Change']*data2_USA_IDX['Population_lag1'])
data2_USA_IDX['Population Effect']=(1/2)*(data2_USA_IDX['CO2 Emission per unit Energy_lag1']*data2_USA_IDX['Energy Intensity_lag1']*data2_USA_IDX['Constant GDP per capita_lag1']*data2_USA_IDX['Population Change']+data2_USA_IDX['CO2 Emission per unit Energy']*data2_USA_IDX['Energy Intensity']*data2_USA_IDX['Constant GDP per capita']*data2_USA_IDX['Population Change'])
data2_USA_IDXb=data2_USA_IDX[['Fuel Mix Effect','Energy Intensity Effect','Income Effect','Population Effect']]
data2_USA_IDXb = data2_USA_IDXb.stack().reset_index()
data2_USA_IDXb.drop(['level_0'],axis=1,inplace=True)
data2_USA_IDXb.columns = ['Effect', 'CO2 Emission']
data2_USA_IDXb['Year']=[
                       1961,1961,1961,1961,
                       1962,1962,1962,1962,
                       1963,1963,1963,1963,
                       1964,1964,1964,1964,
                       1965,1965,1965,1965,
                       1966,1966,1966,1966,
                       1967,1967,1967,1967,
                       1968,1968,1968,1968,
                       1969,1969,1969,1969,
                       1970,1970,1970,1970,
                       1971,1971,1971,1971,
                       1972,1972,1972,1972,
                       1973,1973,1973,1973,
                       1974,1974,1974,1974,
                       1975,1975,1975,1975,
                       1976,1976,1976,1976,
                       1977,1977,1977,1977,
                       1978,1978,1978,1978,
                       1979,1979,1979,1979,
                       1980,1980,1980,1980,
                       1981,1981,1981,1981,
                       1982,1982,1982,1982,
                       1983,1983,1983,1983,
                       1984,1984,1984,1984,
                       1985,1985,1985,1985,
                       1986,1986,1986,1986,
                       1987,1987,1987,1987,
                       1988,1988,1988,1988,
                       1989,1989,1989,1989,
                       1990,1990,1990,1990,
                       1991,1991,1991,1991,
                       1992,1992,1992,1992,
                       1993,1993,1993,1993,
                       1994,1994,1994,1994,
                       1995,1995,1995,1995,
                       1996,1996,1996,1996,
                       1997,1997,1997,1997,
                       1998,1998,1998,1998,
                       1999,1999,1999,1999,
                       2000,2000,2000,2000,
                       2001,2001,2001,2001,
                       2002,2002,2002,2002,
                       2003,2003,2003,2003,
                       2004,2004,2004,2004,
                       2005,2005,2005,2005,
                       2006,2006,2006,2006,
                       2007,2007,2007,2007,
                       2008,2008,2008,2008,
                       2009,2009,2009,2009,
                       2010,2010,2010,2010,
                       2011,2011,2011,2011,
                       2012,2012,2012,2012,
                       2013,2013,2013,2013,
                       2014,2014,2014,2014,
                       2015,2015,2015,2015,
                       2016,2016,2016,2016,
                       2017,2017,2017,2017,
                       2018,2018,2018,2018,
                       2019,2019,2019,2019,
                       2020,2020,2020,2020
                      ]

In [ ]:
chart5a=alt.Chart(data2_CHN_IDXb).mark_bar().encode(
    x=alt.X('Year:O',
            title='Year',
            axis=alt.Axis(titleFontSize=12,labelFontSize=12)),
    y=alt.Y('sum(CO2 Emission)',
            title='Change in CO2 Emission over Previous Year (thousand tonne)',
            axis=alt.Axis(titleFontSize=12,labelFontSize=12)
           ),
    color=alt.Color('Effect:N', 
                    scale = alt.Scale(domain=['Energy Intensity Effect', 'Fuel Mix Effect', 'Income Effect', 'Population Effect'],
                                      range=['lightcoral', 'wheat', 'cornflowerblue', 'mediumturquoise']),
                    legend = alt.Legend(title='Factors under study'))
).properties(width=700, height=350,
            title=alt.TitleParams(text='Contribution of the Four Factors to Annual Change in CO2 Emission: China',
                                   subtitle='(Note: The overall annual change is represented in dotted line)', subtitleFontSize=15,
                                   fontSize=18))
line = alt.Chart(pd.DataFrame({'y': [0]})).mark_rule().encode(y='y')
line2 = alt.Chart(data2_CHN_IDX).mark_line(strokeDash=[3,1]).encode(
    x=alt.X('Year:O',
            title='Year',
            axis=alt.Axis(titleFontSize=12,labelFontSize=12)),
    y=alt.Y('CO2 Change:Q',
            title='Change in CO2 Emission over Previous Year (thousand tonne)',
            axis=alt.Axis(titleFontSize=12,labelFontSize=12)
           ),
    color=alt.value('slategrey')
)
chart5a+line+line2

In [ ]:
chart5b=alt.Chart(data2_USA_IDXb).mark_bar().encode(
    x=alt.X('Year:O',
            title='Year',
            axis=alt.Axis(titleFontSize=12,labelFontSize=12)),
    y=alt.Y('sum(CO2 Emission)',
            title='Change in CO2 Emission over Previous Year (thousand tonne)',
            axis=alt.Axis(titleFontSize=12,labelFontSize=12)
           ),
    color=alt.Color('Effect:N', 
                    scale = alt.Scale(domain=['Energy Intensity Effect', 'Fuel Mix Effect', 'Income Effect', 'Population Effect'],
                                      range=['lightcoral', 'wheat', 'cornflowerblue', 'mediumturquoise']),
                    legend = alt.Legend(title='Factors under study'))
).properties(width=700, height=350,
            title=alt.TitleParams(text='Contribution of the Four Factors to Annual Change in CO2 Emission: US',
                                   subtitle='(Note: The overall annual change is represented in dotted line)', subtitleFontSize=15,
                                   fontSize=18))
line = alt.Chart(pd.DataFrame({'y': [0]})).mark_rule().encode(y='y')
line2 = alt.Chart(data2_USA_IDX).mark_line(strokeDash=[3,1]).encode(
    x=alt.X('Year:O',
            title='Year',
            axis=alt.Axis(titleFontSize=12,labelFontSize=12)),
    y=alt.Y('CO2 Change:Q',
            title='Change in CO2 Emission over Previous Year (thousand tonne)',
            axis=alt.Axis(titleFontSize=12,labelFontSize=12)
           ),
    color=alt.value('slategrey')
)
chart5b+line+line2